# Flight Delay Prediction – Operational Risk for Complex Logistics

## Stage 1: Data Acquisition, Cleaning, and Target Creation

### Objective:
- Load raw data, perform essential cleaning, and establish the primary target variable (Operational Risk Signal). 
-----

## 1. Configuration and Library Imports

In [7]:
import pandas as pd
import numpy as np

# Setting display options
pd.set_option('display.max_columns', 100)

## 2. Data Loading and Initial Inspection


In [8]:
# Load the raw dataset
try:
    df = pd.read_csv('../data/raw/civil_aviation_delay_data.csv', parse_dates=['scheduled_departure', 'actual_departure'])
    print(f"Dataset loaded with {len(df)} rows.")
except FileNotFoundError:
    print("Error: File not found. Ensure 'civil_aviation_delay_data.csv' is in the path '../data/raw/'.")
    df = None

if df is not None:
    # Standardize column names (snake_case)
    df.columns = df.columns.str.lower().str.replace('-', '_')

    print("\nInitial Data Types and Non-Null Values:")
    df.info()

Dataset loaded with 68206 rows.

Initial Data Types and Non-Null Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   scheduled_departure    68206 non-null  datetime64[ns]
 1   scheduled_flight_num   68206 non-null  object        
 2   scheduled_origin       68206 non-null  object        
 3   scheduled_destination  68206 non-null  object        
 4   scheduled_airline      68206 non-null  object        
 5   actual_departure       68206 non-null  datetime64[ns]
 6   actual_flight_num      68205 non-null  object        
 7   actual_origin          68206 non-null  object        
 8   actual_destination     68206 non-null  object        
 9   actual_airline         68206 non-null  object        
 10  day_of_month           68206 non-null  int64         
 11  month                  68206 non-null  int64  

C:\Users\ferna\AppData\Local\Temp\ipykernel_2800\3762231018.py:3: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/civil_aviation_delay_data.csv', parse_dates=['scheduled_departure', 'actual_departure'])


### Dictionary. 
- 'scheduled_departure':    Scheduled date and time of the flight.
- 'scheduled_flight_num':   Scheduled flight number.
- 'scheduled_origin':       Programmed origin city code.
- 'scheduled_destination':  Programmed destination city code.
- 'scheduled_airline':      Scheduled flight airline code.
- 'actual_departure':       Date and time of flight operation.
- 'actual_flight_num':      Flight operation number of the flight.
- 'actual_origin':          Operation origin city code.
- 'actual_destination':     Operation destination city code.
- 'actual_airline':         Airline code of the operated flight.
- 'day_of_month':           Day of the month of flight operation.
- 'month':                  Number of the month of operation of the flight.
- 'year':                   Year of flight operation.
- 'day_of_week_name':       Day of the week of flight operation.
- 'flight_type':            Type of flight, I =International, N =National.
- 'operating_airline':      Name of the airline that operates.
- 'origin_city_name':       Name city of origin.
- 'destination_city_name':  Destination city name.

## 3. Core Feature Creation: Target Variable and Basic Time Features
### 3.1 Target Variable Creation (The Operational Risk Signal)
- Target: 'delay_15' (1 if actual departure time > scheduled departure time + 15 mins)

In [9]:
df['min_diff'] = (df['actual_departure'] - df['scheduled_departure']).dt.total_seconds() / 60
df['delay_15'] = (df['min_diff'] > 15).astype(int)

print(f"\nDistribution of Target 'delay_15':\n{df['delay_15'].value_counts(normalize=True)}")
print(f"Delay rate (Positive Class): {df['delay_15'].mean():.2%}")


Distribution of Target 'delay_15':
delay_15
0    0.81506
1    0.18494
Name: proportion, dtype: float64
Delay rate (Positive Class): 18.49%


The positive class (delay) is minority, confirming the Class Imbalance problem. This requires cost-sensitive learning or class weighting in the modeling stage.

### 3.2 Basic Time Features

In [10]:
df['hour_scheduled'] = df['scheduled_departure'].dt.hour
df['hour_departure'] = df['actual_departure'].dt.hour

In [11]:
df.dtypes

scheduled_departure      datetime64[ns]
scheduled_flight_num             object
scheduled_origin                 object
scheduled_destination            object
scheduled_airline                object
actual_departure         datetime64[ns]
actual_flight_num                object
actual_origin                    object
actual_destination               object
actual_airline                   object
day_of_month                      int64
month                             int64
year                              int64
day_of_week_name                 object
flight_type                      object
operating_airline                object
origin_city_name                 object
destination_city_name            object
min_diff                        float64
delay_15                          int64
hour_scheduled                    int32
hour_departure                    int32
dtype: object

## 4. Save Cleaned Data

In [12]:
# Save the cleaned dataset with the target variable for the next stage.
df.to_csv('../data/interim/01_cleaned_target_data.csv', index=False)
print("\nCleaned data saved to interim/01_cleaned_target_data.csv.")


Cleaned data saved to interim/01_cleaned_target_data.csv.
